In [9]:
import os 
import nibabel as nib
import numpy as np
import pandas as pd

# <span style="color:#FFCC80">In vivo Swiss data processing </span> </br>
Compare and analyze if using optimized parameters improves or deteriorates the final local fields </br>

Keywords: </br>
* sw = slicewise
* pp = pipeline

In [10]:
root_dir = r"E:\msc_data\sc_qsm\swiss_data_mk2\QSM_processing"

In [24]:
# We have 4 subjects inside
dubs = ['hc1', 'hc2', 'sci1', 'sci2']
measurements = ['m1', 'm2', 'm3', 'm4']
bgfr_algos = ['opt_pdf', 'opt_resharp', 'opt_sharp']
rows = []


In [ ]:
# We have 4 subjects inside
#subjects = ['hc1', 'hc2'] #, 'sci1', 'sci2']
#measurements = ['m1', 'm2', 'm3', 'm4']

dubs = ['hc2'] #, 'sci1', 'sci2']
measurements = ['m1', 'm2', 'm3', 'm4']

bgfr_algos = ['opt_lbv', 'def_lbv', 'final_pdf_opt', 'def_pdf', 'opt_resharp',  'def_resharp', 'opt_sharp', 'def_sharp']#,'opt_vsharp_8_to_1','def_iHarperella']

lf_rows = []

In [ ]:
# Now lets collect rows for LF!

for dub in dubs:
    for meas in measurements:
        for algo in bgfr_algos:
            
            # Firsst, define the path pointers
            subj_meas_path = os.path.join(root_dir, dub, meas)
            lf_path = os.path.join(subj_meas_path, 'local_field', algo,'Sepia_localfield.nii.gz')
            if lf_path is None or not os.path.exists(lf_path):
                print(f"Skipping {lf_path} as it does not exist.")
                continue
            gm_msk_path = os.path.join(subj_meas_path, f'custom_{dub}_{meas}_gm_msk.nii.gz')
            wm_msk_path = os.path.join(subj_meas_path, f'custom_{dub}_{meas}_wm_msk.nii.gz')

            # Load data
            lf_img = nib.load(lf_path)
            lf_data = lf_img.get_fdata()

            gm_mask = nib.load(gm_msk_path).get_fdata()
            total_vox_gm = np.sum(gm_mask==1) 
            wm_mask = nib.load(wm_msk_path).get_fdata()
            total_vox_wm = np.sum(wm_mask==1)

            # Compute metrics for GM and WM
            gm_mean = np.mean(lf_data[gm_mask==1])
            gm_std = np.std(lf_data[gm_mask==1])

            wm_mean = np.mean(lf_data[wm_mask==1])
            wm_std = np.std(lf_data[wm_mask==1])

            # Compute how many voxels are in the mask
            gm_nonzero_vox = np.sum(lf_data[gm_mask==1] != 0)
            wm_nonzero_vox = np.sum(lf_data[wm_mask==1] != 0)

            # Compute the contrast metric
            # We want to maximize the contrast between GM and WM
            # WM mean should be negative, if it is positive the contrast will be lower this way
            contrast = np.abs(gm_mean - wm_mean)
            normalizer_denominator = np.sqrt(gm_std**2 + wm_std**2)
            raw_metric = contrast / normalizer_denominator if normalizer_denominator != 0 else 0 # Just in case that the std is 0 - to avoid division by zero

            # Now we penalize if the algo eroded the mask
            gm_penality = gm_nonzero_vox / total_vox_gm 
            wm_penality = wm_nonzero_vox / total_vox_wm

            # Final metric:
            final_metric = raw_metric * gm_penality * wm_penality

            # Now collect row and add to data frame
            lf_rows.append({
                'subject': dub,
                'measurement': meas,
                'algo': algo,
                'mean_gm': gm_mean,
                'std_gm': gm_std,
                'total_vox_gm': total_vox_gm,
                'nonzero_vox_gm': gm_nonzero_vox,
                'mean_wm': wm_mean,
                'std_wm': wm_std,
                'total_vox_wm': total_vox_wm,
                'nonzero_vox_wm': wm_nonzero_vox,
                'contrast factor': contrast,
                'gm_penality': gm_penality,
                'wm_penality': wm_penality,
                'raw_metric': raw_metric,
                'final_metric': final_metric
            })


# Create the data frame
lf_df = pd.DataFrame(lf_rows)

# Save
lf_df.to_csv(os.path.join(root_dir, 'hc2/m1/hc2_m1_bgfr_opt_vs_def_metrics_custom_gm_wm_msk.csv'), index=False)
print(lf_df)

   subject measurement           algo   mean_gm    std_gm  total_vox_gm  \
0      hc2          m1        opt_lbv  0.568873  0.525680           477   
1      hc2          m1        def_lbv  0.196173  0.376231           477   
2      hc2          m1  final_pdf_opt  0.362482  0.455235           477   
3      hc2          m1        def_pdf  0.408185  0.465830           477   
4      hc2          m1    opt_resharp  0.414471  0.444137           477   
5      hc2          m1    def_resharp  0.000000  0.000000           477   
6      hc2          m1      opt_sharp  0.103854  0.336739           477   
7      hc2          m1      def_sharp  0.000000  0.000000           477   
8      hc2          m2        opt_lbv  0.439564  0.492011           496   
9      hc2          m2        def_lbv  0.137170  0.325293           496   
10     hc2          m2  final_pdf_opt  0.273773  0.399865           496   
11     hc2          m2        def_pdf  0.320909  0.413943           496   
12     hc2          m2   

# <span style="color:#16D8E6">Loading and showing the winners</span> </br>

In [9]:
examination_csv_hc1_hc2 =  'bgfr_metrics_custom_gm_wm_msk_dilatedRESHARP.csv' # Select the csv to read
lf_df_hc1_hc2 = pd.read_csv(os.path.join(root_dir,examination_csv_hc1_hc2))
ld_df_hc1_hc2_sorted = lf_df_hc1_hc2.sort_values(by='final_metric', ascending=False)

In [10]:
ld_df_hc1_hc2_sorted.head(5)

,subject,measurement,algo,mean_gm,std_gm,total_vox_gm,nonzero_vox_gm,mean_wm,std_wm,total_vox_wm,nonzero_vox_wm,contrast factor,gm_penality,wm_penality,raw_metric,final_metric
33,hc2,m1,opt_pdf,0.468404,0.470253,477,476,-0.098809,0.467992,2252,2252,0.567214,0.997904,1.000000,0.854957,0.853165
39,hc2,m2,opt_pdf,0.396289,0.433162,496,494,-0.087606,0.445229,2227,2227,0.483895,0.995968,1.000000,0.778999,0.775858
47,hc2,m4,opt_pdf,0.343833,0.449316,494,491,-0.086197,0.450585,2232,2232,0.430030,0.993927,1.000000,0.675800,0.671696
36,hc2,m1,opt_resharp/dilated_mask,0.394375,0.611468,477,477,-0.296662,0.880244,2252,2250,0.691037,1.000000,0.999112,0.644754,0.644181
5,hc1,m2,opt_pdf,0.384207,0.535695,441,440,-0.075465,0.586281,2308,2308,0.459672,0.997732,1.000000,0.578813,0.577501


In [5]:
dilated_test =  'bgfr_metrics_complete_dubs_and_meass_dilated_test.csv' # Select the csv to read
lf_df_hc1_hc2_dil_test= pd.read_csv(os.path.join(root_dir,dilated_test))
ld_df_hc1_hc2_dil_test_sorted= lf_df_hc1_hc2_dil_test.sort_values(by='final_metric', ascending=False)

In [8]:
ld_df_hc1_hc2_dil_test_sorted.head(100)

,subject,measurement,algo,mean_gm,std_gm,total_vox_gm,nonzero_vox_gm,mean_wm,std_wm,total_vox_wm,nonzero_vox_wm,contrast factor,gm_penality,wm_penality,raw_metric,final_metric
17,hc2,m1,opt_pdf,0.378919,0.470250,671,670,-0.079148,0.436276,3264,3264,0.458067,0.998510,1.000000,0.714099,0.713035
23,hc2,m2,opt_pdf,0.341338,0.444211,685,683,-0.071059,0.417333,3240,3240,0.412397,0.997080,1.000000,0.676616,0.674640
20,hc2,m1,opt_resharp/dilated_mask,0.389559,0.756573,671,671,-0.313362,0.964124,3264,3262,0.702921,1.000000,0.999387,0.573562,0.573211
31,hc2,m4,opt_pdf,0.288127,0.457951,700,697,-0.068618,0.421181,3254,3254,0.356745,0.995714,1.000000,0.573375,0.570918
27,hc2,m3,opt_pdf,0.244376,0.462340,653,651,-0.046069,0.413561,3265,3265,0.290445,0.996937,1.000000,0.468222,0.466788
5,hc1,m2,opt_pdf,0.312296,0.754002,647,644,-0.058237,0.689719,3362,3362,0.370532,0.995363,1.000000,0.362600,0.360919
1,hc1,m1,opt_pdf,0.348916,1.140037,645,645,-0.067127,0.753490,3375,3375,0.416042,1.000000,1.000000,0.304449,0.304449
18,hc2,m1,opt_resharp,0.380682,0.570080,671,618,-0.028245,0.426189,3264,1677,0.408927,0.921013,0.513787,0.574514,0.271863
9,hc1,m3,opt_pdf,0.284571,1.040017,623,623,-0.051134,0.870087,3413,3413,0.335706,1.000000,1.000000,0.247574,0.247574
13,hc1,m4,opt_pdf,0.216303,0.767831,636,635,-0.038948,0.728996,3346,3346,0.255251,0.998428,1.000000,0.241082,0.240703


In [37]:
ld_df_hc1_hc2_dil_test_sorted_contrast = lf_df_hc1_hc2_dil_test.sort_values(by='contrast factor', ascending=False)
ld_df_hc1_hc2_dil_test_sorted_contrast.head(5)

,subject,measurement,algo,mean_gm,std_gm,total_vox_gm,nonzero_vox_gm,mean_wm,std_wm,total_vox_wm,nonzero_vox_wm,contrast factor,gm_penality,wm_penality,raw_metric,final_metric
20,hc2,m1,opt_resharp/dilated_mask,0.389559,0.756573,671,671,-0.313362,0.964124,3264,3262,0.702921,1.000000,0.999387,0.573562,0.573211
0,hc1,m1,opt_lbv,0.530615,0.589751,645,438,0.058260,0.342314,3375,1300,0.472355,0.679070,0.385185,0.692707,0.181190
17,hc2,m1,opt_pdf,0.378919,0.470250,671,670,-0.079148,0.436276,3264,3264,0.458067,0.998510,1.000000,0.714099,0.713035
10,hc1,m3,opt_resharp,0.382815,3.171093,623,593,-0.052051,1.555972,3413,1770,0.434866,0.951846,0.518605,0.123113,0.060772
1,hc1,m1,opt_pdf,0.348916,1.140037,645,645,-0.067127,0.753490,3375,3375,0.416042,1.000000,1.000000,0.304449,0.304449


# <span style="color:#b338ff">BGFR default vs phantom optimized using all measurements from hc2 </span> </br>


In [28]:
cnr_sorted_hc2_m1_bgfr_def_and_opt_comp = pd.read_csv(os.path.join(root_dir, 'hc2/m1/hc2_m1_bgfr_opt_vs_def_metrics_custom_gm_wm_msk.csv')).sort_values(by='final_metric', ascending=False)
cnr_sorted_hc2_m1_bgfr_def_and_opt_comp

,subject,measurement,algo,mean_gm,std_gm,total_vox_gm,nonzero_vox_gm,mean_wm,std_wm,total_vox_wm,nonzero_vox_wm,contrast factor,gm_penality,wm_penality,raw_metric,final_metric
2,hc2,m1,final_pdf_opt,0.362482,0.455235,477,476,-0.076146,0.342733,2252,2252,0.438629,0.997904,1.000000,0.769755,0.768141
3,hc2,m1,def_pdf,0.408185,0.465830,477,476,-0.087131,0.452097,2252,2252,0.495316,0.997904,1.000000,0.763028,0.761428
11,hc2,m2,def_pdf,0.320909,0.413943,496,493,-0.072102,0.438168,2227,2227,0.393011,0.993952,1.000000,0.652000,0.648056
10,hc2,m2,final_pdf_opt,0.273773,0.399865,496,493,-0.062138,0.336049,2227,2227,0.335911,0.993952,1.000000,0.643111,0.639221
26,hc2,m4,final_pdf_opt,0.275051,0.394915,494,455,-0.060598,0.328757,2232,2232,0.335649,0.921053,1.000000,0.653207,0.601638
27,hc2,m4,def_pdf,0.311548,0.424923,494,455,-0.070626,0.441942,2232,2232,0.382174,0.921053,1.000000,0.623363,0.574150
18,hc2,m3,final_pdf_opt,0.228829,0.382900,472,472,-0.047216,0.315572,2261,2261,0.276045,1.000000,1.000000,0.556336,0.556336
19,hc2,m3,def_pdf,0.263727,0.427013,472,472,-0.055367,0.425741,2261,2261,0.319095,1.000000,1.000000,0.529189,0.529189
4,hc2,m1,opt_resharp,0.414471,0.444137,477,438,-0.046043,0.316350,2252,1155,0.460514,0.918239,0.512877,0.844539,0.397730
0,hc2,m1,opt_lbv,0.568873,0.525680,477,422,0.001258,0.341791,2252,1065,0.567615,0.884696,0.472913,0.905251,0.378743


In [34]:
std_sorted_hc2_m1_bgfr_def_and_opt_comp = pd.read_csv(os.path.join(root_dir, 'hc2/m1/hc2_m1_bgfr_opt_vs_def_metrics_custom_gm_wm_msk.csv')).sort_values(by='std_wm', ascending=True)
std_sorted_hc2_m1_bgfr_def_and_opt_comp = std_sorted_hc2_m1_bgfr_def_and_opt_comp[std_sorted_hc2_m1_bgfr_def_and_opt_comp["mean_gm"]!=0]
std_sorted_hc2_m1_bgfr_def_and_opt_comp

,subject,measurement,algo,mean_gm,std_gm,total_vox_gm,nonzero_vox_gm,mean_wm,std_wm,total_vox_wm,nonzero_vox_wm,contrast factor,gm_penality,wm_penality,raw_metric,final_metric
25,hc2,m4,def_lbv,0.158449,0.343495,494,177,-0.005549,0.168751,2232,330,0.163998,0.358300,0.147849,0.428520,0.022701
17,hc2,m3,def_lbv,0.132023,0.309535,472,168,-0.012300,0.173382,2261,346,0.144323,0.355932,0.153030,0.406789,0.022157
1,hc2,m1,def_lbv,0.196173,0.376231,477,170,-0.014059,0.180717,2252,335,0.210233,0.356394,0.148757,0.503692,0.026704
9,hc2,m2,def_lbv,0.137170,0.325293,496,176,-0.018685,0.183110,2227,334,0.155856,0.354839,0.149978,0.417519,0.022219
22,hc2,m3,opt_sharp,0.054429,0.285077,472,433,-0.009966,0.222081,2261,1170,0.064394,0.917373,0.517470,0.178194,0.084591
30,hc2,m4,opt_sharp,0.075836,0.280310,494,452,-0.018981,0.227176,2232,1131,0.094817,0.914980,0.506720,0.262791,0.121840
14,hc2,m2,opt_sharp,0.081567,0.302564,496,451,-0.017071,0.237566,2227,1136,0.098638,0.909274,0.510103,0.256413,0.118930
6,hc2,m1,opt_sharp,0.103854,0.336739,477,438,-0.022833,0.249365,2252,1155,0.126687,0.918239,0.512877,0.302342,0.142386
20,hc2,m3,opt_resharp,0.264047,0.406865,472,433,-0.034632,0.294904,2261,1170,0.298679,0.917373,0.517470,0.594384,0.282162
28,hc2,m4,opt_resharp,0.336524,0.405347,494,452,-0.033037,0.298360,2232,1131,0.369561,0.914980,0.506720,0.734255,0.340429
